# Numerical Solution of Differential Equations

It is very common that we need to solve differential equations in science and engineering. The broad class of "differential equations" comprises many types of initial value problems and boundary value problems, both of which might involve ordinary or partial differential equations.  Here, we will focus only on numerical solutions of initial value problems in the form of ordinary differential equations or systems of ordinary differential equations.

The general idea with an initial value problem is that we have information about the rate of change of a quantity of interest (i.e., we know the derivative of a function), we know the starting value of that quantity of interest, and we would like to, based only on information about starting value and the derivative, predict the quantity of interest at some future time, position, Temperature, or other independent variable that describes our system.

We'll do a very basic demonstration of how this works, but it is based on the idea that if you know the derivative, you can use the value of the deriviative to estimate how much your dependent variable (e.g., "y") will change over a small step in the independent variable (e.g., "x"). The most basic approach is the Euler method, which is basically just an algebraic rearrangement of the classic definition of a slope (m) of a function:

$$m = \frac{dy}{dx} \approx \frac{\Delta y}{\Delta x}$$

Rearranging:

$$\Delta y = m\cdot \Delta x$$

Substituting discrete values of y and x in for the deltas:

$$y_1 - y_0 = \frac{dy}{dx} \cdot(x_1 - x_0)$$

We start at our initial value of (x0,y0); then, we take a small, numerical step in $x$, $\Delta x$ (let's call it $x_{step}$), and we update our value of the variable of interest, y, based on the derivative at $x_i$, which we assume is constant over the step h and thus extrapolate over the whole step:

$$y_1 = y_0 + \frac{dy}{dx} \cdot x_{step}$$

Generally, we'll write it as an iterative loop in terms of $x_i$ and $x_{i+1}$

$$y_{i+1} = y_i + \frac{dy}{dx} \cdot x_{step}$$

Let's test this method on a first order ordinary differential equation:

$$\frac{dy}{dx} = -y \ \ \textrm{with} \ \ y(0) = 10$$

We know that the analytical solution to this initial value problem is the following exponential function:

$$y(x) = 10 e^{-x}$$

Plot the solution to this IVP, y(x), over the domain x = [0, 6] in order to visualize how y changes as a function of x. Note, I called my function "ytrue(x)", and I evaluated it on a vector of x-values from 0 to 6 called "xtrue."  If you see those labels subsequently in this worksheet, it just means that I am using the analytical solution to this initial value problem, $y(x) = 10e^{-x}$

    ytrue(x) = 10*exp.(-x)
    xtrue = collect(range(0, stop = 6, length = 100);

If we *didn't* know how to solve that equation, or if we could otherwise not solve it analytically, we would need to use a numerical method. The Euler method is the most simple way to numerically solve a differential equation. We'll implement it to solve this example using a while loop.  I've also added in a bit of visualization to show how the Euler step works.

    using Plots
    xstep = 0.75                #I get to choose the size of the step for the independent variable
    x = 0                       #initial x
    y = 10                      #initial y
    xlist = [0.]                 #I'm going to store all x values in a vector; initializing that
    ylist = [10.]                #Initialize a vector containing y values for each x value

     while xlist[end] < 6        #I want to continue iterating until I reach x = 6
         dydx = -y               #derivative of y with respect to x   
         y    =  y + xstep*dydx  #update y value using an Euler step
         x    =  x + xstep       #update x value using step size
         append!(xlist, x)       #append new value of x
         append!(ylist, y)       #append new value of y
         plt = plot(xtrue, ytrue(xtrue)) #plot true solution
         for i = 1:length(xlist)-1
             scatter!([xlist[i], xlist[i+1]], [ylist[i], ylist[i+1]], markercolor = :red, legend = nothing)
         end
         display(plt)
         sleep(0.5)                #pause for 0.5 seconds before next iteration
     end

Generally speaking, this is how a numerical differential equation solver works. It calculates the slope(s) at the current values for all function arguments, it takes a step in independent variable, and it predicts the value of the function based on the slope. With the step size I chose above (xstep = 0.75), you can see that the Euler method does a very bad job of approximating the true solution. You can probably already see the reason for this:  the Euler Method calculates the value of the derivative at $x_i$, and then it assumes that this derivative value is constant over the entire step in x, but we know this isn't true here (nor for most differential equations that we need to use numerical methods for). You can improve the accuracy of the Euler Method by taking smaller steps; go ahead and experiment to figure out how small of a step you need to make before you are satisified that the numerical solution is roughly the same as the analytical solution.  Supress the looped plotting to keep it a bit more tidy.

The Euler method is the simplest numerical method for solving differential equations, and it is generally limited because it needs such a small step size. There are far more sophisticated algoritms for numerical solution of differential equations, but the ones we'll consider in this course are mostly based on the same concept. They are make iterative steps in independent variable and update the dependent variable at each step.  It would be worth your time to at least read up on Explicit Runge Kutta Methods - these are very commonly used for numerical solution of ODEs. Higher order Runge-Kutta methods are available, and they do a good job at forecasting how the deriviate will change over the $x_{step}$, and so they can approximate the true solution much more accurately and with much larger step sizes than an Euler method.

https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods

As usual, the methods in Julia are more sophisticated than what we would usually write. They are flexible and powerful, and they can handle almost any type of differential equation you'd throw at them. Beyond writing your own loops to learn more about how the numerical method works, I would always defer to using the Julia packages for numerical solution of ODEs. For this class, we will use the excellent DifferentialEquations.jl package (which also has good documention below). 

https://diffeq.sciml.ai/stable/

Let's start by installing the package:

    using Pkg
    Pkg.add("DifferentialEquations")

To perform a numerical solution of an ODE or system of ODEs in Julia, we need to write a function. In the most basic and Matlab-like version of Julia-syntax, This will be a multivariate, vector function, very similar to what we did with nlsolve and LeastSquaresOptim. It should be of the form f(dependent variabe, parameters, independent variable), where dependent variable is a vector of all state variables in the ODE system, and it should return a vector of derivatives for each dependent variable with respect to the independent variable.

The basic rules:

1.  The first argument, "u", should be a 1D array that contains all of the dependent variables in the system of ODEs that you are solving. For a single ODE, that argument will be a scalar. 
2. The second argument, "p", is an array that contains any parameters you wish to pass to the ODE system.
3. The third argument to the function should be a scalar representing the independent variable that describes your system. As far as the ODE solver is concerned, it is always a scalar. Remember that these loops work iteratively, and at any given time, there is only a single value of the independent variable that you are considering. 
4. The function should return the deriviative of each depedent variable with respect to the single independent variable in a vector-like structure. If it is a single ODE, simply return a scalar.

For the simple first order example above, this would be an appropriate way to format your function to pass to the ODE solver:

    function f(y, p, x)      #function should be of form f(dependent variable, parameters, independent variable)
        D = -y               #Calculate the value of the derivative of y at the current value of x
    end

To solve this *initial value problem* numerically, we'll first create an OdeProblem. To do this we need the initial state of the system (i.e., initial conditions), the span of integration, and the function that contains our system of ODEs:

    using DifferentialEquations
    y0 = 10
    tspan = (0.0, 6.0) #integration range; 2-element array of floats; first and last points
    prob  = ODEProblem(f, y0, tspan)
    sol   = solve(prob)
       
This will return a solution structure, which has multiple fields as usual that you can access with the struturename.fieldname syntax. To see the list of fields, run:

    fieldnames(typeof(sol))

Lots of detail available here:

https://diffeq.sciml.ai/stable/basics/solution/

The ones of most interest to us are usually sol.t (independent variable values) and sol.u (state/dependent variable values).  

    xvals = sol.t
    yvals = sol.u
    plot(xvals, yvals)
    scatter!(xvals,yvals)

You may notice from that the solver did not take even steps in x, and perhaps it skipped over x steps that you were interested in seeing. That is fine - some solvers use adaptive step sizes.  In general, they are far better at deciding on a good step size than we are.  This is why we only specify the starting and ending point for integration.  The solver decides on the intermediate points, not us.  That said, you may wish to get higher resolution and view the solution to the differential equation at additional points in the domain where you solved it. 

If all you need is a nice, smooth plot, you can just plot the solution structure directly.  It will produce a solution on a fine x-grid and plot it:

    plot(sol)
    
It is beautiful!

ODE solution structures are also nice because they create an interpolating polynomial by default.  This provides you with a function that you can use to generate values of your state variable (y in this case) at specific values of your independent variable (x in this case).  So let's say I want to see the solution at the values of x in the array below:

    x_new = [0, 0.25, 0.75, 1.0, 1.25, 1.75, 3, 3.16, 4, 4.05, 4.1, 4.75, 5, 5.5, 6.0]
    
I can pass them directly to the solution structure and get the output at those points.  

    y_new = sol(x_new)

This is stellar!

Let's plot it next to the original solution output just to show that they are different sets:

    plot(sol, label = "y(t)")
    scatter!(sol.t, sol.u, label = "sol.u")
    scatter!(x_new, y_new, label = "sol(xnew)")

As usual, you can change the options for your ode solvers; this is done using keyword or positional arguments of varying types.  Be aware that there are a dizzying array of options available for DifferentialEquations.jl.  We'll use them as the need arises, but for the most part, default options will be OK for us.

The only other thing that we'll mention is changing the solver itself. For CEN587, most of the time, using default options will be fine.  Sometimes, you will encounter *stiff* systems of differential equations. In high level terms, what that means is you have two or more coupled dependent variables that are changing as functions of the indepdendent variable. If those two variables change on extremely different scales, then you have what is called a stiff system, and they require specialized solvers.  When might you see this?  When the derivative of one variable is much, much larger than the derivative of the other variable.  In these cases, you may need to your solver.  Julia has a lot of options here.

https://diffeq.sciml.ai/stable/solvers/ode_solve/

Try defaults, and if they don't work (and your model is correctly coded), you probably have a stiff system.  This will usually manifest as the default solver taking a very long time to solve the ODE system. If you see this type of behavior, and you think you have a stiff system, try Rodas5() or Rosenbrock23(). It is super easy to change the solver.  You just add the solver name to the solve() command:

    solve(prob, Rodas5())
    solve(prob, Rosenbrock23())

Alternatively, you can just give solve a hint that says your system is stiff using a keyword argument alg_hints:

    solve(prob, alg_hints = "stiff")


How can you tell when you need to change solvers?  Usually the default option is a good choice.  If you notice that it is taking a long time to run, making an unusually large number of steps, or failing to converge altogether, you probably are solving a stiff system.  

**Systems of ODES**

We covered a lot of ground in terms of how to use the solvers above, but all of the examples we have considered so far deal with one ODE that we can easily solve by hand. Numerical ODE solvers really shine when you have a very complex differential equation that does not have an analytical solution, or when you have coupled systems of nonlinear differential equations, which are generally either very difficult or impossible to solve analytically. By "coupled" differential equations, I mean that the derivatives of the dependent variables are functions of each other (see system below). 

\begin{align*}
\frac{dx}{dt} &= -x^2 + y^2 \\
\frac{dy}{dt} &= -xy \\
x(0) &= 0 \\
y(0) &= 3 \\
\end{align*}

A typical question we might ask about this system is **what are the values of x and y at t = 10?**

I see that dx/dt is a function of x and y, and dy/dt is also a function of x and y.  I cannot solve one without solving the other. Though it is sometimes possible to solve a coupled system analytically, the vast majority of cases must be handled with numerical methods.  Fortunately, the framework we established above for solving a single differential equation is very general, and it is super easy to extend it to systems of differential equations. 

We just need to formulate our system of differential equations as multivariate vector functions of the form f(y,p,t) or, in other words, f(dependent variables, parameters, independent variable). Both of our dependent variables, (x and y in this case), need to be passed to the function in a single, vector-like structure. The function also needs to return dx/dt and dy/dt in an array. This is very similar to what we saw with **nlsolve** and **LeastSquaresOptim.optimize**. For this example, the following is an acceptable way to define the above system of ODEs so that it calculates the derivate of each state variable (x and y) with respect to the independent variable (time) and returns dx/dt and dy/dt in a 2-element vector that is the same size as u.

    function diffs(u, p, t)
        x    = u[1]
        y    = u[2]
        D1 = -x^2 + y^2
        D2 = -x*y
        D  = [D1, D2]
        return D
    end

Now that the function is defined, we can integrate it on the domain t = [0, 10].  We follow the same conventions as above.  The span of the independent variable can be a two element vector, and the initial values of x and y must be passed to the function as an array.  It is worth digging into the solution structure.  The independent variable values in sol.t are a straightforward array, but the state variables in sol.u are returned as an array of arrays, which may be unfamiliar if you're coming from something like Matlab.  But, if you need to separate those values out for some reason, it's easy to do. See Solution handling for more detail, but I show how to do it in the last lines of this bit of code.

https://diffeq.sciml.ai/stable/basics/solution/

    var0  = [0., 3.]
    tspan = (0., 10.)
    prob2 = ODEProblem(diffs, var0, tspan)
    sol2 = solve(prob2)
    tvals = sol2.t
    xvals = sol2[1,:]   #extracts timeseries for 1st state variable
    yvals = sol2[2,:]   #extracts timeseries for 2nd state variable

If I just want a nice plot, I'd just run:

    plot(sol2)

But separating out the state variables does allow you to work with them independently:

    plot(tvals, xvals, linecolor = :black)
    plot!(tvals, yvals, linecolor = :blue, xlabel = "time", ylabel = "value")
    
Not necessarily for plotting, but it will be useful elsewhere in the course, so learn how to extract individual components of interest from your solution structures now.

It may often be useful to you to pass extra parameters into your differential equation model.  The solver will only operate on the independent variable and dependent variable (i.e., the first two arguments).  If you need to pass additional parameters or arguments to your ODE function, you need to do this using an anonymous fucntion. Let's change that system just a bit to illustrate this concept:

\begin{align*}
\frac{dx}{dt} &= -ax^2 + by^2 \\
\frac{dy}{dt} &= -cxy \\
x(0) &= 0 \\
y(0) &= 3 \\
\end{align*}

Where a, b, and c are constant coefficients.  For the values (a,b,c) = (0.2, 1.3, 8), what are the values of x and y at t = 10?  What about for (a,b,c) = (1.5, 0.3, -0.1)?

    function diff2(var, p, t)
        x = var[1]
        y = var[2]
        a, b, c = p
        dxdt = -a*x^2 + b*y^2;
        dydt = -c*x*y;
        D = [dxdt, dydt] #This a perfectly acceptable way to create a 1D array of derivatives.
    end

    tspan = (0., 10.)   #Integration span
    x0    = 0
    y0    = 3
    var0  = [x0, y0]  #initial values for x and y
    a     = 0.2
    b     = 1.3
    c     = 8
    p     = (a, b, c)

    prob3 = ODEProblem(diff2, var0, tspan, p)
    sol3  = solve(prob3)
    plot(sol3, xlabel = "time", ylabel = "value", label = ["x" "y"])